In [1]:
import os
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras import layers
import tensorflow_addons as tfa
import tqdm
from tensorflow import keras
from scipy import stats


2.6.2


In [9]:
df = pd.read_csv("./top_5_soft_clusters.csv")
# # n_features = 300
# print(df.head())

# # features = [f'f_{i}' for i in range(n_features)]
# # feature_columns = ['investment_id', 'time_id'] + features

train = pd.read_parquet("/home/aimlproject1/train_low_mem.parquet")
print(train['investment_id'].unique().max())
# labels= pd.read_csv('./label_.csv')
# train = train.drop('row_id' ,axis =1)
# train = train.merge(labels , on = "investment_id")


3773


In [10]:
df.head(20)

,Unnamed: 0,1,2,3,4,5
0,0,4,0,9,8,7
1,1,4,0,8,9,7
2,2,4,9,7,8,6
3,3,4,0,9,8,7
4,4,4,0,9,8,7
5,6,4,0,9,8,7
6,7,4,0,9,8,7
7,8,3,2,1,5,8
8,9,4,0,9,8,7
9,10,4,0,9,8,7


In [5]:
# df.drop()
# df.head()
df1 = pd.read_csv("./soft_cluster.csv");
df1.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9
0,0,0.054595,0.031438,0.029102,0.031205,0.661284,0.032620,0.034505,0.038554,0.041547,0.045151
1,1,0.083935,0.032117,0.029322,0.030915,0.229369,0.032084,0.032866,0.035085,0.040294,0.040275
2,2,0.038829,0.032452,0.030487,0.033432,0.629864,0.036930,0.040791,0.048694,0.048499,0.060023
3,3,0.060418,0.027939,0.025707,0.027279,0.698007,0.028047,0.029139,0.031715,0.035199,0.036549
4,4,0.032289,0.017632,0.016289,0.017420,0.809912,0.018194,0.019148,0.021236,0.023107,0.024774


In [7]:
df.head()
# train.head(-10)
# train['label_feature'].unique()
# # arr = []
# for index, g in train.iterrows():
#       model = keras.models.load_model("model1_swish")
#       # print(g)
#       # print(g.index)
#       col = g.index
#       g = g.to_numpy().reshape(1,-1)
#       # print(g)
#       g = pd.DataFrame(g,columns = col)
#       g.pop("time_id")
#       g.pop("target")
#       i = g.pop("label_feature").values[0]
#       investment_id = g.pop("investment_id")
#       print([investment_id, g])
#       # print(g.columns)
#       pred = model.predict([float(x) for x in [investment_id, g]])
#       arr.append(pred[0][0])
#       break
# print(arr)

,Unnamed: 0,1,2,3,4,5
0,0,4,0,9,8,7
1,1,4,0,8,9,7
2,2,4,9,7,8,6
3,3,4,0,9,8,7
4,4,4,0,9,8,7


In [3]:
# 1 - swish 


def get_model(activ):
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation=activ)(investment_id_x)
    investment_id_x = layers.Dense(64, activation=activ)(investment_id_x)
    investment_id_x = layers.Dense(64, activation=activ)(investment_id_x)
    # "swish" tfa.activations.lisht
    feature_x = layers.Dense(256, activation=activ)(features_inputs)
    feature_x = layers.Dense(256, activation=activ)(feature_x)
    feature_x = layers.Dense(256, activation=activ)(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dense(512, activation=activ, kernel_regularizer="l2")(x)
    x = layers.Dense(128, activation=activ, kernel_regularizer="l2")(x)
    x = layers.Dense(32, activation=activ, kernel_regularizer="l2")(x)
    output = layers.Dense(1)(x)
    
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_dir = ['saved_model0' , 'saved_model1' , 'saved_model2' , 'saved_model3', 'saved_model4', 'saved_model5']

checkpoints = []

for i in  checkpoint_dir : 
    checkpoint = ModelCheckpoint(filepath=i, 
                            frequency = "epoch",
                            save_weights_only = False,
                             verbose= True)
    checkpoints.append(checkpoint)

dict = {}
copy = train
predicted = []
dd  = []
ii= []
gg= []
from scipy import stats
for i,g in (train.groupby('label_feature')):  
    if i!=-1:  
        print("Running cluster %f\n" , i) 
        g = g.loc[:, g.columns != 'label_feature']
        investment_id = g.pop("investment_id")
        time_id = g.pop("time_id")
        y = g.pop("target")
        dd = y
        gg = g
        investment_ids = list(investment_id.unique())
        investment_id_size = len(investment_ids) + 1
        investment_id_lookup_layer = layers.IntegerLookup(max_tokens=investment_id_size)
        with tf.device("gpu"):
            investment_id_lookup_layer.adapt(investment_id)
        # dict[i] = get_model()
        
        model = get_model(tfa.activations.lisht)
        model.fit([investment_id, g],y, epochs=5  ,callbacks= [checkpoints[i]])
        # joblib.dump(model, './finalized_model.sav')
        # # y_vals = []
        # predicted = model.predict([investment_id, g])
        # ii = investment_id
        print(i)
        
            

In [4]:
# dd = dd.to_numpy()
# model = get_model()
# model.load_weights("MOD")
# pred = list(model.predict([ii,gg]))
# dd = list(dd)
# # (dd.shape)
from tensorflow.keras.models import load_model
# # dd.reshape(-1,1)
# predicted = list(predicted)
# # print(pearson_score)
c = "saved_model0"
for i,g in (train.groupby('label_feature')):
            if i == 0:
                  g = g.loc[:, g.columns != 'label_feature']
                  investment_id = g.pop("investment_id")
                  time_id = g.pop("time_id")
                  y = g.pop("target")
                  
                  loaded_model = load_model(c)
                #   investment_ids = list(investment_id.unique())
                #   investment_id_size = len(investment_ids) + 1
                #   investment_id_lookup_layer = layers.IntegerLookup(max_tokens=investment_id_size)
                #   with tf.device("gpu"):
                #       investment_id_lookup_layer.adapt(investment_id)
                  
                #   model = get_model()
                  # name = "/home/aimlproject1/home/"+ 'model' + str(i)
                  # model = keras.models.load_model("model1_swish")
                  # model = get_model()
                  # model = tf.saved_model.load(name)
                  # tf.keras.load(name)
                  pred = list(loaded_model.predict([investment_id, g]))
                  print(pred)
                  hehe = np.squeeze(np.asarray(y))
                  haha = np.squeeze(np.asarray(pred))
                  result = np.all(haha == haha[0])
                  if result:
                        print(i)       
                  pearson_score = stats.pearsonr(haha, hehe)[0]
                  print(i,pearson_score)
                  print("\n")
# 3 0.08607929178188765(swish) , 

2022-03-31 23:47:40.603912: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-31 23:47:40.655434: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-31 23:47:40.655757: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-31 23:47:40.657224: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

InternalError:  Attempting to perform BLAS operation using StreamExecutor without BLAS support
	 [[node model_12/dense_123/MatMul (defined at tmp/ipykernel_147922/4026103426.py:32) ]] [Op:__inference_predict_function_2965]

Function call stack:
predict_function


In [ ]:
soft = pd.read_csv("soft_cluster.csv")

In [ ]:
soft.head(10)

In [ ]:
len((np.sort(train['investment_id'].unique())))